# PSNR and SSIM

In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim

def calculate_psnr(image1, image2):

    mse = np.mean((image1 - image2) ** 2)
    if mse == 0:
        return float('inf')
    max_pixel = 255.0
    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

def calculate_ssim_color(image1, image2):

    # Ensure the images have a color channel (e.g., RGB)
    if image1.shape[-1] != 3 or image2.shape[-1] != 3:
        raise ValueError("Input images must have 3 color channels (e.g., RGB).")

    # Determine the minimum dimension and set a suitable win_size
    min_dim = min(image1.shape[0], image1.shape[1])
    win_size = min(7, min_dim - 1) if min_dim > 1 else 1

    # Calculate SSIM for multichannel images
    ssim_value, _ = ssim(image1, image2, full=True, channel_axis=-1, win_size=win_size)
    return ssim_value

In [ ]:
def divide_image_vertically(image, slices=4):

    height, width, channels = image.shape
    slice_width = width // slices
    sliced_images = []

    for i in range(slices):
        start_x = i * slice_width
        end_x = (i + 1) * slice_width if i != slices - 1 else width
        sliced_images.append(image[:, start_x:end_x])

    return sliced_images

In [ ]:
import os

# Listing mock filenames to demonstrate the matching process.
file_list = [
    "reconstruction_gs-063085_e-000037_b-000000.png",
    "samples_cfg_scale_9.00_gs-063085_e-000037_b-000000.png",
    "reconstruction_gs-063085_e-000038_b-000001.png",
    "samples_cfg_scale_9.00_gs-063085_e-000038_b-000001.png",
    "reconstruction_gs-063085_e-000039_b-000002.png",
    "samples_cfg_scale_9.00_gs-063085_e-000039_b-000002.png"
]

def match_reconstruction_and_sample_images(file_list):

    reconstruction_image_paths = []
    sample_image_paths = []

    for file in file_list:
        if file.startswith("reconstruction_"):
            # Extract key (e.g., "gs-063085_e-000037_b-000000")
            key = "_".join(file.split("_")[1:4])
            # Find matching sample image
            matching_sample = next(
                (f for f in file_list if f.startswith("samples_") and key in f), None
            )
            if matching_sample:
                reconstruction_image_paths.append(file)
                sample_image_paths.append(matching_sample)

    return reconstruction_image_paths, sample_image_paths


sample_paths,reconstruction_paths = match_reconstruction_and_sample_images(file_list)


['reconstruction_gs-063085_e-000037_b-000000.png', 'reconstruction_gs-063085_e-000038_b-000001.png', 'reconstruction_gs-063085_e-000039_b-000002.png']


In [11]:
# # running examples
# sample_paths =['samples1.png','samples2.png','samples3.png','samples4.png','samples5.png','samples6.png']
# reconstruction_paths=['reconstruction1.png','reconstruction2.png', 'reconstruction3.png','reconstruction4.png','reconstruction5.png','reconstruction6.png']

In [ ]:

for i in range(len(sample_paths)):
    print(f"Index: {i}")
    image_path_1 = sample_paths[i]
    image_path_2 = reconstruction_paths[i]

    # Read images using OpenCV
    image1 = cv2.imread(image_path_1)
    image2 = cv2.imread(image_path_2)

    # Ensure images have the same dimensions
    if image1.shape != image2.shape:
        raise ValueError("Input images must have the same dimensions.")

    # Divide both images into 4 slices
    image1_slices = divide_image_vertically(image1, slices=4)
    image2_slices = divide_image_vertically(image2, slices=4)

    # Compare each slice and calculate PSNR
    psnr_scores = []
    ssim_scores = []
    for i, (slice1, slice2) in enumerate(zip(image1_slices, image2_slices)):
        psnr_value = calculate_psnr(slice1, slice2)
        psnr_scores.append(psnr_value)
        print(f"Slice {i + 1} PSNR: {psnr_value:.4f} dB")

        ssim_value = calculate_ssim_color(slice1, slice2)
        ssim_scores.append(ssim_value)
        print(f"Slice {i + 1} SSIM: {ssim_value:.4f} dB")



    # # Calculate PSNR
    # psnr_value = calculate_psnr(image1, image2)
    # print(f"PSNR: {psnr_value} dB")

    # # Calculate SSIM
    # ssim_value = calculate_ssim_color(image1, image2)
    # print(f"SSIM: {ssim_value}")
